In [27]:
import pandas as pd  # Импортируем библиотеку pandas для работы с табличными данными
import requests
import time
from bs4 import BeautifulSoup
# Загружаем CSV-файл с данными об ATC-категориях для диабетических препаратов
df = pd.read_csv('ATC_categories_diabetis.csv', index_col=0)

# Извлекаем информацию о лекарственных средствах (их названия) и ATC-коды
drugs = df['ATC_info'].values  # Получаем список названий препаратов

# Выводим количество загруженных препаратов
print(len(drugs))

# Список названий препаратов
drugs

173


array(['Calcium glucoheptonate', 'Teplizumab', 'Tolrestat',
       'Bexagliflozin', 'Luseogliflozin', 'Sotagliflozin',
       'Ipragliflozin', 'Ertugliflozin', 'Empagliflozin', 'Canagliflozin',
       'Dapagliflozin', 'Beinaglutide', 'Semaglutide', 'Dulaglutide',
       'Albiglutide', 'Lixisenatide', 'Liraglutide', 'Exenatide',
       'Dorzagliatin', 'Carfloglitazar', 'Tirzepatide', 'Imeglimin',
       'Mitiglinide', 'Benfluorex', 'Pramlintide', 'Nateglinide',
       'Repaglinide', 'Guar gum', 'Teneligliptin',
       'Gemigliptin and rosuvastatin', 'Evogliptin',
       'Sitagliptin and simvastatin', 'Gemigliptin', 'Linagliptin',
       'Alogliptin', 'Saxagliptin', 'Vildagliptin', 'Sitagliptin',
       'Lobeglitazone', 'Pioglitazone', 'Rosiglitazone', 'Troglitazone',
       'Voglibose', 'Miglitol', 'Acarbose',
       'Sitagliptin and dapagliflozin', 'Metformin and teneligliptin',
       'Metformin, linagliptin and empagliflozin',
       'Metformin and lobeglitazone',
       'Metformin, 

In [28]:
import pandas as pd  # Работа с таблицами
import requests
import time

# Загружаем CSV с ATC-категориями
df = pd.read_csv('ATC_categories_diabetis.csv', index_col=0)

# Получаем названия препаратов
drugs = df['ATC_info'].values
print(f"Количество загруженных препаратов: {len(drugs)}")

# Функция для получения RxCUI по названию препарата
def get_rxcui(drug_name):
    url = f"https://rxnav.nlm.nih.gov/REST/rxcui.json?name={drug_name}&search=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("idGroup", {}).get("rxnormId", [None])[0]
    return None

# Функция для получения ATC-кода по RxCUI
def get_atc_code(rxcui):
    url = f"https://rxnav.nlm.nih.gov/REST/rxclass/class/byRxcui.json?rxcui={rxcui}&relaSource=ATC"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        atc_codes = set()  # Set убирает дубликаты
        if "rxclassDrugInfoList" in data and "rxclassDrugInfo" in data["rxclassDrugInfoList"]:
            for item in data["rxclassDrugInfoList"]["rxclassDrugInfo"]:
                atc_code = item["rxclassMinConceptItem"]["classId"]
                atc_name = item["rxclassMinConceptItem"]["className"]
                atc_codes.add(f"{atc_code} ({atc_name})")
        return ", ".join(sorted(atc_codes)) if atc_codes else None
    return None

# Функция для получения данных о лекарстве
def get_drug_data(drug_name, rxcui):
    drug_info = {"Drug Name": drug_name, "RxCUI": rxcui, "ATC Code": get_atc_code(rxcui)}

    # Получаем ингредиенты (IN/MIN)
    url_ing = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/related.json?tty=IN"
    response_ing = requests.get(url_ing)
    if response_ing.status_code == 200:
        data = response_ing.json()
        ingredients = [item["name"] for group in data.get("relatedGroup", {}).get("conceptGroup", []) for item in group.get("conceptProperties", [])]
        drug_info["Ingredients"] = ", ".join(ingredients)

    # Получаем клинические дозировки (SCDG)
    url_scdg = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/related.json?tty=SCDG"
    response_scdg = requests.get(url_scdg)
    if response_scdg.status_code == 200:
        data = response_scdg.json()
        scdg = [item["name"] for group in data.get("relatedGroup", {}).get("conceptGroup", []) for item in group.get("conceptProperties", [])]
        drug_info["Clinical Dose Groups"] = ", ".join(scdg)

    # Получаем группы форм дозировки (DFG)
    url_dfg = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/related.json?tty=DFG"
    response_dfg = requests.get(url_dfg)
    if response_dfg.status_code == 200:
        data = response_dfg.json()
        dfg = [item["name"] for group in data.get("relatedGroup", {}).get("conceptGroup", []) for item in group.get("conceptProperties", [])]
        drug_info["Dose Form Groups"] = ", ".join(dfg)

    # Получаем брендированные группы дозировки (SBDG)
    url_sbdg = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/related.json?tty=SBDG"
    response_sbdg = requests.get(url_sbdg)
    if response_sbdg.status_code == 200:
        data = response_sbdg.json()
        sbdg = [item["name"] for group in data.get("relatedGroup", {}).get("conceptGroup", []) for item in group.get("conceptProperties", [])]
        drug_info["Branded Dose Groups"] = ", ".join(sbdg)

    return drug_info

# Список для хранения данных о всех препаратах
all_drugs_data = []

# Запускаем цикл по всем препаратам
for drug_name in drugs:
    print(f"🔍 Обрабатываем: {drug_name}")
    rxcui = get_rxcui(drug_name)
    if rxcui:
        drug_data = get_drug_data(drug_name, rxcui)
        all_drugs_data.append(drug_data)
    time.sleep(1)  # Пауза между запросами

# Создаем DataFrame и сохраняем в CSV
df_results = pd.DataFrame(all_drugs_data)
df_results.to_csv("drug_data_results.csv", index=False)


Количество загруженных препаратов: 173
🔍 Обрабатываем: Calcium glucoheptonate
🔍 Обрабатываем: Teplizumab
🔍 Обрабатываем: Tolrestat
🔍 Обрабатываем: Bexagliflozin
🔍 Обрабатываем: Luseogliflozin
🔍 Обрабатываем: Sotagliflozin
🔍 Обрабатываем: Ipragliflozin
🔍 Обрабатываем: Ertugliflozin
🔍 Обрабатываем: Empagliflozin
🔍 Обрабатываем: Canagliflozin
🔍 Обрабатываем: Dapagliflozin
🔍 Обрабатываем: Beinaglutide
🔍 Обрабатываем: Semaglutide
🔍 Обрабатываем: Dulaglutide
🔍 Обрабатываем: Albiglutide
🔍 Обрабатываем: Lixisenatide
🔍 Обрабатываем: Liraglutide
🔍 Обрабатываем: Exenatide
🔍 Обрабатываем: Dorzagliatin
🔍 Обрабатываем: Carfloglitazar
🔍 Обрабатываем: Tirzepatide
🔍 Обрабатываем: Imeglimin
🔍 Обрабатываем: Mitiglinide
🔍 Обрабатываем: Benfluorex
🔍 Обрабатываем: Pramlintide
🔍 Обрабатываем: Nateglinide
🔍 Обрабатываем: Repaglinide
🔍 Обрабатываем: Guar gum
🔍 Обрабатываем: Teneligliptin
🔍 Обрабатываем: Gemigliptin and rosuvastatin
🔍 Обрабатываем: Evogliptin
🔍 Обрабатываем: Sitagliptin and simvastatin
🔍 Обраба

In [40]:
df_results

,Drug Name,RxCUI,ATC Code,Ingredients,Clinical Dose Groups,Dose Form Groups,Branded Dose Groups
0,Calcium glucoheptonate,47618,A12AA (Calcium),calcium gluceptate,,,
1,Teplizumab,2621880,A10XX (Other drugs used in diabetes),teplizumab,teplizumab Injectable Product,Injectable Product,Tzield Injectable Product
2,Tolrestat,38386,A10XA (Aldose reductase inhibitors),tolrestat,,,
3,Bexagliflozin,2627044,A10BK (Sodium-glucose co-transporter 2 (SGLT2)...,bexagliflozin,"bexagliflozin Oral Product, bexagliflozin Pill","Oral Product, Pill","Bexacat Oral Product, Bexacat Pill, Brenzavvy ..."
4,Sotagliflozin,2638675,A10BK (Sodium-glucose co-transporter 2 (SGLT2)...,sotagliflozin,"sotagliflozin Oral Product, sotagliflozin Pill","Oral Product, Pill","Inpefa Oral Product, Inpefa Pill"
...,...,...,...,...,...,...,...
107,Cyclobarbital,2978,"N05CA (Barbiturates, plain)",cyclobarbital,,,
108,Ketazolam,28181,N05BA (Benzodiazepine derivatives),ketazolam,,,
109,Benorilate,1372,N02BA (Salicylic acid and derivatives),benorilate,"benorilate Oral Liquid Product, benorilate Ora...","Oral Product, Pill, Oral Liquid Product",
110,Papaveretum,253192,N02AA (Natural opium alkaloids),papaveretum,,,


In [ ]:
df_results.to_csv("all_drug_info.csv", index=False)